## Load Embedded Data


In [1]:
import pandas
import pickle

MODEL ="dunzhang/stella_en_400M_v5" #dunzhang/stella_en_1.5B_v5
DIFFICULTY = "hard"
SAMPLE = 500

save_dir = f"embeddings/{MODEL}/{DIFFICULTY}/{SAMPLE}"

hotpot_qa_df = pandas.read_pickle(f'{save_dir}/df.pkl')
with open(f'{save_dir}/contexts.pkl', 'rb') as f:contexts = pickle.load(f)
with open(f'{save_dir}/similarity_matrix.pkl', 'rb') as f:similarity_matrix = pickle.load(f)

similarity_matrix = similarity_matrix.numpy()


# Agglomerative Clustering

In [2]:
from clustering.algorithms import Agglomerative
from clustering.analysis import ClusterAnalysis
from clustering.visualization.counts import plot_cluster_counts, plot_avg_cluster_variance,plot_graphs,plot_mean_avg_cluster_similarity

clusters  = Agglomerative(similarity_matrix,contexts,0.68)

cluster_counts_plot = plot_cluster_counts(clusters,
                                          "Agglomerative Clustering",
                                          "Cluster No.",
                                          "No. of Items in Cluster")

cluster_analysis = ClusterAnalysis(clusters,similarity_matrix)

avg_cluster_variance = cluster_analysis.get_avg_variance_for_each_cluster()
mean_avg_cluster_sim = cluster_analysis.get_mean_avg_cluster_similarity_for_each_cluster()

avg_cluster_variance_plots= plot_avg_cluster_variance(avg_variance_dict = avg_cluster_variance,
                                                      graph_name        = "Average Variance in Each Cluster",
                                                      x_label           = "Cluster No.",
                                                      y_label           = "Avg variance in Cluster")
mean_avg_cluster_sim_plots= plot_mean_avg_cluster_similarity(mean_avg_cluster_sim_dict = mean_avg_cluster_sim,
                                                             graph_name                = "Mean Average Similarity in Each Cluster",
                                                             x_label                   = "Cluster No.",
                                                             y_label                   = "Mean Avg similarity in Cluster")

plot_graphs([cluster_counts_plot,avg_cluster_variance_plots,mean_avg_cluster_sim_plots])

In [8]:
import statistics

def get_avg_cluster_variance(cluster):
    variances = []
    for i, node in enumerate(cluster):
        similarities = []
        print(f"Cosine similarity between node {node} and nodes: ")
        for j, other_node in enumerate(cluster):
            if i != j: 
                cosine_sim = similarity_matrix[i][j]
                similarities.append(cosine_sim)
                print(f"                                             {other_node} = {cosine_sim}")
        variance = statistics.variance(similarities)
        variances.append(variance)
        print(f"                             average similarity = {statistics.mean(similarities)}")
        print(f"                                       variance = {variance}")
        print()

    return statistics.mean(variances)

def get_avg_cluster_similarity(cluster:list[int]):
    avg_similarities = []
    for i, node in enumerate(cluster):
        similarities = []
        for j, other_node in enumerate(cluster):
            if i != j: 
                cosine_sim = similarity_matrix[i][j]
                similarities.append(cosine_sim)
        avg_similarities.append(statistics.mean(similarities))
    
    return statistics.mean(avg_similarities)

num = 228
cluster = clusters[num]
print(cluster,"\n")
x = get_avg_cluster_variance(cluster)
print(f"average cluster variance        = {x}")
print(f"mean average cluster similarity = {get_avg_cluster_similarity(cluster)}")

[1762, 1765, 1768, 1769, 1770, 1771, 2759, 2760] 

Cosine similarity between node 1762 and nodes: 
                                             1765 = 0.6498722434043884
                                             1768 = 0.7966527342796326
                                             1769 = 0.5945354700088501
                                             1770 = 0.6721450090408325
                                             1771 = 0.41524097323417664
                                             2759 = 0.7579242587089539
                                             2760 = 0.5065721273422241
                             average similarity = 0.6275632381439209
                                       variance = 0.01814681850373745

Cosine similarity between node 1765 and nodes: 
                                             1762 = 0.6498722434043884
                                             1768 = 0.5755731463432312
                                             1769 = 0.5900585055351257
  

In [4]:
from typing import Dict
from clustering.graph.create_from_clusters import ClusterGraph
import networkx as nx

cluster_graph = ClusterGraph(clusters=clusters,similarity_matrix=similarity_matrix)

individual_cluster_graphs:Dict[int, nx.Graph] = cluster_graph.get_individual_cluster_graphs()
connected_graph:nx.Graph = cluster_graph.get_connected_graph()

In [5]:
print(nx.is_connected(connected_graph))

True


In [6]:
import plotly
from clustering.graph.visualization.graph_plots import get_individual_cluster_graph_plots, get_connected_graph_plot

graph_plots:Dict[int,plotly.graph_objs.Figure] = get_individual_cluster_graph_plots(individual_cluster_graphs,contexts)
cluster_plot:plotly.graph_objs.Figure = get_connected_graph_plot(connected_graph,clusters)

In [7]:
import pickle

with open(f'clustering/visualization/graph_plots.pkl', 'wb') as file:
    pickle.dump(graph_plots, file)

with open(f'clustering/visualization/cluster_plot.pkl', 'wb') as file:
    pickle.dump(cluster_plot, file)

# python cluster_plots.py --cluster_plot_dir="clusters/cluster_plot" --graph_plots_dir="clusters/graph_plots"

In [22]:
!python cluster_plots.py --cluster_plot_dir="cluster_plot" --graph_plots_dir="graph_plots"

python: can't open file 'c:\\Users\\nikhi\\OneDrive\\Desktop\\RAG-Evaluator-HotPotQA\\cluster_plots.py': [Errno 2] No such file or directory


In [ ]:
from clustering.visualization.cluster_plots import ClusterVisualization

plot = ClusterVisualization(cluster_plot,graph_plots)
plot.run()

In [ ]:
from scipy.cluster import hierarchy

# Create a linkage matrix
Z = hierarchy.linkage(dissimilarity_matrix, method='single')
print("done")
# Create a dendrogram
fig = plt.figure(figsize=(10, 5))

In [ ]:
GROQ_API_KEY = "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"

In [ ]:

dn = hierarchy.dendrogram(Z)